In [46]:
# Libraries
import pandas as pd

In [47]:
# Upload of the datasets
df_total = pd.read_csv('raw_data/eurostat_import/eurostat_totals.csv')
df_products = pd.read_csv('raw_data/eurostat_import/eurostat_products.csv')

In [48]:
df_products.head()

,REPORTER_LAB,PARTNER_LAB,PRODUCT,PRODUCT_LAB,FLOW_LAB,STAT_PROCEDURE_LAB,PERIOD_LAB,INDICATORS_LAB,INDICATOR_VALUE
0,Austria,Ukraine,2710,Petroleum oils and oils obtained from bitumino...,IMPORT,Total,Jan.-Dec. 2021,VALUE_IN_EUR,105076
1,Austria,Ukraine,2701,"Coal; briquettes, ovoids and similar solid fue...",IMPORT,Total,Jan.-Dec. 2023,VALUE_IN_EUR,8584475
2,Austria,Ukraine,2701,"Coal; briquettes, ovoids and similar solid fue...",IMPORT,Total,Jan.-Dec. 2021,VALUE_IN_EUR,24
3,Austria,Ukraine,72,IRON AND STEEL,IMPORT,Total,Jan.-Dec. 2023,VALUE_IN_EUR,1247827
4,Austria,Ukraine,72,IRON AND STEEL,IMPORT,Total,Jan.-Dec. 2021,VALUE_IN_EUR,11321048


In [49]:
## 1. Drop Useless columns: FLOW_LAB, STAT_PROCEDURE_LAB, INDICATORS_LAB
df_products = df_products.drop(['FLOW_LAB', 'STAT_PROCEDURE_LAB', 'INDICATORS_LAB', 'PRODUCT_LAB'], axis=1)
df_total = df_total.drop(['FLOW_LAB', 'STAT_PROCEDURE_LAB', 'INDICATORS_LAB', 'PRODUCT_LAB'], axis=1)

In [50]:
## 2. Convert the column PERIOD_LAB extracting and keeping only the year
df_products['PERIOD_LAB'] = df_products['PERIOD_LAB'].astype(str).str.extract(r'(\d{4})')
df_total['PERIOD_LAB'] = df_total['PERIOD_LAB'].astype(str).str.extract(r'(\d{4})')

In [51]:
## 3. Changes PARTNER_LAB column: from "All countries of the world" to "World"
df_products['PARTNER_LAB'] = df_products['PARTNER_LAB'].replace('All countries of the world', 'World')
df_total['PARTNER_LAB'] = df_total['PARTNER_LAB'].replace('All countries of the world', 'World')

In [52]:
## 4. Cleaning of the REPORTER_LAB
unique_values = df_products['REPORTER_LAB'].unique()
print(unique_values)

['Austria' "Belgium (incl. Luxembourg 'LU' -> 1998)" 'Bulgaria' 'Cyprus'
 'Czechia' "Germany (incl. German Democratic Republic 'DD' from 1991)"
 'Denmark' 'Estonia' "Spain (incl. Canary Islands 'XB' from 1997)"
 'European Union - 27 countries (AT, BE, BG, CY, CZ, DE, DK, EE, ES, FI, FR, GR, HR, HU, IE, IT, LT, LU, LV, MT, NL, PL, PT, RO, SE, SI, SK)'
 'Finland'
 "France (incl. Saint Barthélemy 'BL' -> 2012; incl. French Guiana 'GF', Guadeloupe 'GP', Martinique 'MQ', Réunion 'RE' from 1997; incl. Mayotte 'YT' from 2014)"
 'Greece' 'Croatia' 'Hungary' 'Ireland (Eire)'
 "Italy (incl. San Marino 'SM' -> 1993)" 'Lithuania' 'Luxembourg' 'Latvia'
 'Malta' 'Netherlands' 'Poland' 'Portugal' 'Romania' 'Sweden' 'Slovenia'
 'Slovakia']


In [53]:
# Definition of the dictionary to clean the names in the dataset
dict = {
    "Belgium (incl. Luxembourg 'LU' -> 1998)" : "Belgium",
    "Germany (incl. German Democratic Republic 'DD' from 1991)" : "Germany",
    "Spain (incl. Canary Islands 'XB' from 1997)" : "Spain",
    "European Union - 27 countries (AT, BE, BG, CY, CZ, DE, DK, EE, ES, FI, FR, GR, HR, HU, IE, IT, LT, LU, LV, MT, NL, PL, PT, RO, SE, SI, SK)" : "European Union",
    "France (incl. Saint Barthélemy 'BL' -> 2012; incl. French Guiana 'GF', Guadeloupe 'GP', Martinique 'MQ', Réunion 'RE' from 1997; incl. Mayotte 'YT' from 2014)" : "France",
    "Ireland (Eire)" : "Ireland",
    "Italy (incl. San Marino 'SM' -> 1993)" : "Italy"
}

In [54]:
# Dictionary application
df_products['REPORTER_LAB'] = df_products['REPORTER_LAB'].replace(dict)
df_total['REPORTER_LAB'] = df_total['REPORTER_LAB'].replace(dict)

In [55]:
# Dictionary definition (following the code of the previous chart)
dict_HS4 = {
    'Crude/Refined Petroleum' : ['2709', '2710'],
    'Natural Gas' : ['2711'],
    'Coal' : ['2701'],
    'Seed Oils' : ['1512']
}

dict_HS2 = {
    'Metals' : ['72','74','75','76','73','78','79','80','81','83','82','59'],
    'Precious stones, metals, & pearls' : ['71'],
    'Machines' : ['84','85'],
    'Chemicals' : ['31','28','29','38','30','33','34','32','36','35','37'],
    'Cereals' : ['10'],
    'Oils Seeds & Oleaginous Fruits' : ['12'],
    'Minerals' : ['26', '25'],
    'Wood' : ['44']
}

In [56]:
# Added of a Section column
df_products['Section'] = ""

In [57]:
# Dictionary flatten
def flatten_dict(d):
    return {val: key for key, val_list in d.items() for val in val_list}

map_hs4 = flatten_dict(dict_HS4)
map_hs2 = flatten_dict(dict_HS2)

# Assignment of the sections following the dictionaries
def assegna_section_da_product(df):
    df = df.copy()
    df['PRODUCT'] = df['PRODUCT'].astype(str)
    match_hs4 = df['PRODUCT'].map(map_hs4)
    match_hs2 = df['PRODUCT'].map(map_hs2)
    df['Section'] = match_hs4.fillna(match_hs2)
    return df

In [58]:
# New dataset with the sections
df_products = assegna_section_da_product(df_products)

In [59]:
# Grouped by REPORTER_LAB, PARTNER_LAB, PRODUCT, PERIOD_LAB, Section, summing the value on INDICATOR_VALUE
df_grouped_products = df_products.groupby(['Section', 'REPORTER_LAB', 'PARTNER_LAB', 'PERIOD_LAB'])['INDICATOR_VALUE'].sum().reset_index()
df_grouped_products['PARTNER_LAB'] = df_grouped_products['PARTNER_LAB'].replace('Russian Federation (Russia)', 'Russia')

In [60]:
# Obtain the final datasets
df_grouped_products.to_csv('../../data/final/trade-data/final_datasets/output_data_v2.csv', index=False)
df_total.to_csv('../../data/final/trade-data/final_datasets/output_data_total_v2.csv', index=False)